<a href="https://colab.research.google.com/github/Sumetjutha/MyProject/blob/main/4_Build_A_Stock_Prediction_PTTEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install quandl

In [74]:
# This program Predicts stock prices by nusing machine learning models
# Install the dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas_datareader import data
import seaborn as sns
import matplotlib.pyplot as plt

In [76]:
df = data.DataReader('PTTEP.bk',data_source="yahoo")
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,58.00,55.25,57.00,55.75,5657100.0,44.283619
2016-01-05,57.25,54.00,57.00,54.00,8163500.0,42.893543
2016-01-06,54.50,52.75,54.50,54.00,7304800.0,42.893543
2016-01-07,51.75,48.50,51.00,48.75,14233900.0,38.723339
2016-01-08,51.25,48.50,49.25,50.25,11249900.0,39.914829


In [77]:
# Get the Adjusted Close Price
df = df[['Adj Close']]

In [78]:
# Take a look at the new data
print(df.head())

            Adj Close
Date                 
2016-01-04  44.283619
2016-01-05  42.893543
2016-01-06  42.893543
2016-01-07  38.723339
2016-01-08  39.914829


In [79]:
print(df.tail())

            Adj Close
Date                 
2020-12-24      95.75
2020-12-25      96.50
2020-12-28      95.00
2020-12-29      97.75
2020-12-30      98.25


In [80]:
# A variable for prediction 'n' days out into the future
forecast_out = 1

In [81]:
# Create anothr column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)

In [82]:
print(df.head())

            Adj Close  Prediction
Date                             
2016-01-04  44.283619   42.893543
2016-01-05  42.893543   42.893543
2016-01-06  42.893543   38.723339
2016-01-07  38.723339   39.914829
2016-01-08  39.914829   38.723339


In [83]:
print(df.tail())

            Adj Close  Prediction
Date                             
2020-12-24      95.75       96.50
2020-12-25      96.50       95.00
2020-12-28      95.00       97.75
2020-12-29      97.75       98.25
2020-12-30      98.25         NaN


In [84]:
### Create the Independent data set (X)
# convert the data frame to a numpy array
X = np.array(df.drop(['Prediction'],1))

In [85]:
# Remove the last 'n' rows 
X = X[:-forecast_out]

In [86]:
print(X)

[[44.28361893]
 [42.89354324]
 [42.89354324]
 ...
 [96.5       ]
 [95.        ]
 [97.75      ]]


In [87]:
### Create the independent data set (y)
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])

In [88]:
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]

In [89]:
print(y)

[42.89354324 42.89354324 38.72333908 ... 95.         97.75
 98.25      ]


In [90]:
# Split the data into 80% training and 20% testting
x_train,x_test,y_train,y_test = train_test_split(X, y ,test_size=0.3)

In [91]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)

In [92]:
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [93]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test,y_test)

In [94]:
print("svm confidence: ", svm_confidence)

svm confidence:  0.9938322843223614


In [95]:
# Create and trainLinear Regression Model
lr = LinearRegression()

In [96]:
# Train the model
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [97]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
lr_confidence = lr.score(x_test,y_test)

In [98]:
print("lr confidence: ", lr_confidence)

lr confidence:  0.9950396429996194


In [101]:
# Set x_forecast equal to the last 30 rows of the original ata set from Adj Close columns
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

In [102]:
print(x_forecast)

[[98.25]]


In [103]:
# Print the predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)

In [104]:
print(lr_prediction)

[98.30764449]


In [105]:
# Print the predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)

In [106]:
print(svm_prediction)

[98.50443508]
